In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("../data/archive/AviationData.csv", encoding='cp1252', low_memory=False)
# US_State_Codes = pd.read_csv('../data/archive/USState_Codes.csv', encoding='cp1252', low_memory=False)

In [3]:
# Data Cleaning 
df.columns = df.columns.str.replace('.', ' ', regex=True)
df.columns = df.columns.str.title()
# dropping rows that are non-Airplane related
df = df[(df['Aircraft Category'] == 'Airplane')]
# dropping rows that are Amateur-Built aircraft
df = df[(df['Amateur Built'] == 'No')]
# capitalizing all Make strings to avoid double categories
df['Make'] = df['Make'].str.title() 
# capitalizing all Air Carrier strings to avoid double categories
df['Air Carrier'] = df['Air Carrier'].str.title() 
# capitalizing all Air Carrier strings to avoid double categories
df['Weather Condition'] = df['Weather Condition'].str.upper() 

# dropping unimportant columns, mostly ID numbers
df.drop(axis=1, inplace=True, labels=['Aircraft Category', 'Airport Code', 'Publication Date', 'Weather Condition',
                                      'Airport Name', 'Amateur Built', 'Event Id', 'Accident Number', 'Registration Number', 'Latitude', 'Longitude'])
df.head()

,Investigation Type,Event Date,Location,Country,Injury Severity,Aircraft Damage,Make,Model,Number Of Engines,Engine Type,Far Description,Schedule,Purpose Of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Broad Phase Of Flight,Report Status
5,Accident,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Mcdonnell Douglas,DC9,2.0,Turbo Fan,Part 129: Foreign,SCHD,NaN,Air Canada,NaN,NaN,1.0,44.0,Climb,Probable Cause
7,Accident,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Part 91: General Aviation,NaN,Personal,NaN,0.0,0.0,0.0,2.0,Takeoff,Probable Cause
8,Accident,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Part 91: General Aviation,NaN,Business,NaN,0.0,0.0,0.0,2.0,Landing,Probable Cause
12,Accident,1982-01-02,"HOMER, LA",United States,Non-Fatal,Destroyed,Bellanca,17-30A,1.0,Reciprocating,Part 91: General Aviation,NaN,Personal,NaN,0.0,0.0,1.0,0.0,Cruise,Probable Cause
13,Accident,1982-01-02,"HEARNE, TX",United States,Fatal(1),Destroyed,Cessna,R172K,1.0,Reciprocating,Part 91: General Aviation,NaN,Personal,NaN,1.0,0.0,0.0,0.0,Takeoff,Probable Cause


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24417 entries, 5 to 88886
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation Type      24417 non-null  object 
 1   Event Date              24417 non-null  object 
 2   Location                24411 non-null  object 
 3   Country                 24410 non-null  object 
 4   Injury Severity         23604 non-null  object 
 5   Aircraft Damage         23147 non-null  object 
 6   Make                    24414 non-null  object 
 7   Model                   24399 non-null  object 
 8   Number Of Engines       21866 non-null  float64
 9   Engine Type             20461 non-null  object 
 10  Far Description         23937 non-null  object 
 11  Schedule                2936 non-null   object 
 12  Purpose Of Flight       20734 non-null  object 
 13  Air Carrier             10356 non-null  object 
 14  Total Fatal Injuries    21651 non-null

In [5]:
# df['Make'].value_counts() 

In [6]:
# df['Model'].value_counts()

In [7]:
# df['Number Of Engines'].value_counts() 
# possible correlation with aircraft damage/injury

In [8]:
df['Far Description'].value_counts()


091                               13409
Part 91: General Aviation          5077
NUSN                               1122
NUSC                                804
137                                 742
121                                 653
135                                 540
Part 137: Agricultural              360
Part 135: Air Taxi & Commuter       233
UNK                                 228
129                                 202
Part 121: Air Carrier               161
PUBU                                125
Part 129: Foreign                    78
Non-U.S., Non-Commercial             72
Non-U.S., Commercial                 67
Unknown                              18
091K                                 14
Public Use                           12
125                                   5
Part 125: 20+ Pax,6000+ lbs           5
ARMF                                  4
Part 133: Rotorcraft Ext. Load        2
Part 91 Subpart K: Fractional         1
Public Aircraft                       1


In [9]:
# df[df['Far Description' 'Model']]

df['Far Description'] = df['Far Description'].str.replace('091', 'Part 91: General Aviation')
df['Far Description'] = df['Far Description'].str.replace('Part 91: General AviationK', 'Part 91: General Aviation')
df['Far Description'] = df['Far Description'].str.replace('137', 'Part 137: Agricultural')
df['Far Description'] = df['Far Description'].str.replace('121', 'Part 121: Air Carrier')
df['Far Description'] = df['Far Description'].str.replace('135', 'Part 135: Air Taxi & Commuter')
df['Far Description'] = df['Far Description'].str.replace('091', 'Part 91: General Aviation')
df['Far Description'] = df['Far Description'].str.replace('129', 'Part 129: Foreign')

df['Far Description'].value_counts()



Part 91: General Aviation                                  18500
NUSN                                                        1122
NUSC                                                         804
Part 137: Agricultural                                       742
Part 121: Air Carrier                                        653
Part 135: Air Taxi & Commuter                                540
Part Part 137: Agricultural: Agricultural                    360
Part Part 135: Air Taxi & Commuter: Air Taxi & Commuter      233
UNK                                                          228
Part 129: Foreign                                            202
Part Part 121: Air Carrier: Air Carrier                      161
PUBU                                                         125
Part Part 129: Foreign: Foreign                               78
Non-U.S., Non-Commercial                                      72
Non-U.S., Commercial                                          67
Unknown                  

In [10]:
# df['Schedule'].value_counts()

# only three values Scheduled, Unscheduled, Unknown
# replace strings for more clarity

In [11]:
df['Purpose Of Flight'].value_counts()
# get rid of aerial applications and most of the other categories

Personal                     13339
Instructional                 3120
Aerial Application            1066
Unknown                        838
Business                       764
Positioning                    350
Aerial Observation             170
Skydiving                      166
Ferry                          163
Other Work Use                 155
Executive/corporate            148
Flight Test                    120
Banner Tow                      89
Public Aircraft - Federal       52
Air Race show                   48
Public Aircraft                 42
Glider Tow                      35
Public Aircraft - State         24
Firefighting                    17
Public Aircraft - Local         12
ASHO                             5
Air Race/show                    4
Air Drop                         3
PUBS                             3
External Load                    1
Name: Purpose Of Flight, dtype: int64

In [12]:
df['Air Carrier'].value_counts() 
# comparison with frequency of accidents

Pilot                    174
American Airlines         47
United Airlines           43
Southwest Airlines Co     43
Delta Air Lines           42
                        ... 
Highlight Sport Llc        1
Olmsted Aaron Jr           1
Uni-Air                    1
Oklahoma Aviation Llc      1
Gerber Richard E           1
Name: Air Carrier, Length: 8508, dtype: int64

In [13]:
# df['Broad Phase Of Flight'].value_counts()
